# Tworzenie bazy

In [1]:
import sqlite3

# Połączenie z bazą danych SQLite
conn = sqlite3.connect('db_zapasy.db')
c = conn.cursor()

# Tworzenie tabeli Dostawcy
c.execute('''CREATE TABLE Dostawcy (
                id_dostawca INTEGER PRIMARY KEY,
                nazwa_dostawcy TEXT NOT NULL
            )''')

# Tworzenie tabeli Produkty
c.execute('''CREATE TABLE Produkty (
                kod_produktu INTEGER PRIMARY KEY,
                nazwa_produktu TEXT NOT NULL,
                kategoria TEXT,
                zapas_bezpieczenstwa INTEGER,
                ilosc_szt_w_opk_zbiorczym INTEGER,
                objetosc_sztuki DECIMAL(5, 2),
                wymiar_wysokosc INTEGER,
                wymiar_szerokosc INTEGER,
                wymiar_glebokosc INTEGER,
                wymiar_masa INTEGER
            )''')

# Tworzenie tabeli Magazyny
c.execute('''CREATE TABLE Magazyny (
                id_magazyn INTEGER PRIMARY KEY,
                nazwa_magazynu TEXT NOT NULL,
                pojemnosc INTEGER
            )''')

# Tworzenie tabeli StanyMagazynowe
c.execute('''CREATE TABLE StanyMagazynowe (
                id_stan INTEGER PRIMARY KEY,
                data_stanu DATE,
                kod_produktu INTEGER,
                id_magazyn INTEGER,
                ilosc_na_stanie INTEGER,
                ilosc_zarezerwowana INTEGER,
                ilosc_dostepna INTEGER,
                FOREIGN KEY (kod_produktu) REFERENCES Produkty(kod_produktu),
                FOREIGN KEY (id_magazyn) REFERENCES Magazyny(id_magazyn)
            )''')

# Tworzenie tabeli Klienci
c.execute('''CREATE TABLE Klienci (
                id_klient INTEGER PRIMARY KEY,
                nazwa_klienta TEXT NOT NULL
            )''')

# Tworzenie tabeli Zamowienia
c.execute('''CREATE TABLE Zamowienia (
                id_zamowienia INTEGER PRIMARY KEY,
                id_klient INTEGER,
                data_zamowienia DATE,
                oczekiwany_termin_wysylki DATE,
                data_wysylki DATE,
                status_wysylki TEXT,
                kanal_sprzedazy TEXT,
                FOREIGN KEY (id_klient) REFERENCES Klienci(id_klient)
            )''')

# Tworzenie tabeli ZamowieniaSzczegoly
c.execute('''CREATE TABLE ZamowieniaSzczegoly (
                zamowienie_produkt_id INTEGER PRIMARY KEY,
                id_zamowienia INTEGER,
                kod_produktu INTEGER,
                ilosc INTEGER,
                FOREIGN KEY (id_zamowienia) REFERENCES Zamowienia(id_zamowienia),
                FOREIGN KEY (kod_produktu) REFERENCES Produkty(kod_produktu)
            )''')

# Tworzenie tabeli Dostawy
c.execute('''CREATE TABLE Dostawy (
                id_dostawy INTEGER PRIMARY KEY,
                id_dostawca INTEGER,
                data_zamowienia DATE,
                oczekiwana_data_dostawy DATE,
                data_dostawy DATE,
                status_dostawy TEXT,
                FOREIGN KEY (id_dostawca) REFERENCES Dostawcy(id_dostawca)
            )''')

# Tworzenie tabeli DostawySzczegoly
c.execute('''CREATE TABLE DostawySzczegoly (
                id_szczegol_dostawy INTEGER PRIMARY KEY,
                id_dostawy INTEGER,
                kod_produktu INTEGER,
                ilosc INTEGER,
                FOREIGN KEY (id_dostawy) REFERENCES Dostawy(id_dostawy),
                FOREIGN KEY (kod_produktu) REFERENCES Produkty(kod_produktu)
            )''')

# Zatwierdzenie zmian i zamknięcie połączenia
conn.commit()
conn.close()

print("Baza danych i tabele zostały utworzone pomyślnie.")


Baza danych i tabele zostały utworzone pomyślnie.


# Aktualizacja bazy

In [3]:
import sqlite3
import pandas as pd
import os

def display_added_records(num_added_records, table_name):
    print(f"Do tabeli '{table_name}' dodano {num_added_records} nowych rekordów.")

def add_new_records(conn, excel_file, table_name):
    # Pobranie istniejących rekordów z danej tabeli jako DataFrame
    existing_data = pd.read_sql_query(f"SELECT * FROM {table_name}", conn)

    # Wczytanie danych z pliku Excel z folderu "dane"
    new_data = pd.read_excel(excel_file)

    # Wybierz tylko te rekordy z pliku Excel, które jeszcze nie istnieją w bazie danych
    new_records = new_data[~new_data.iloc[:, 0].isin(existing_data.iloc[:, 0])]

    num_added_records = len(new_records)

    # Dodaj nowe rekordy do bazy danych
    new_records.to_sql(table_name, conn, if_exists='append', index=False)

    return num_added_records

# Połączenie z bazą danych SQLite
conn = sqlite3.connect('DB_ZAPASY.db')

# Lista plików Excel i odpowiadających tabel SQL
excel_files = {
    'Dostawcy.xlsx': 'Dostawcy',
    'Produkty.xlsx': 'Produkty',
    'Magazyny.xlsx': 'Magazyny',
    'StanyMagazynowe.xlsx': 'StanyMagazynowe',
    'Klienci.xlsx': 'Klienci',
    'Zamówienia.xlsx': 'Zamowienia',
    'ZamówieniaSzczegóły.xlsx': 'ZamowieniaSzczegoly',
    'Dostawy.xlsx': 'Dostawy',
    'DostawySzczegóły.xlsx': 'DostawySzczegoly'
}

# Dodaj nowe rekordy z każdego pliku Excel do odpowiednich tabel SQL
for excel_file, table_name in excel_files.items():
    excel_file_path = os.path.join("Dane", excel_file)
    if os.path.exists(excel_file_path):
        num_added_records = add_new_records(conn, excel_file_path, table_name)
        display_added_records(num_added_records, table_name)
    else:
        print(f"Plik '{excel_file}' nie istnieje. Pomijanie...")

# Zamknięcie połączenia z bazą danych
conn.close()


Do tabeli 'Dostawcy' dodano 0 nowych rekordów.
Do tabeli 'Produkty' dodano 0 nowych rekordów.
Do tabeli 'Magazyny' dodano 0 nowych rekordów.
Do tabeli 'StanyMagazynowe' dodano 0 nowych rekordów.
Do tabeli 'Klienci' dodano 0 nowych rekordów.
Do tabeli 'Zamowienia' dodano 0 nowych rekordów.
Do tabeli 'ZamowieniaSzczegoly' dodano 0 nowych rekordów.
Do tabeli 'Dostawy' dodano 0 nowych rekordów.
Do tabeli 'DostawySzczegoly' dodano 0 nowych rekordów.


# Wyciąganie danych

In [7]:
import pandas as pd
# Połączenie z bazą danych SQLite
conn = sqlite3.connect('DB_ZAPASY.db')

# Pobranie danych z tabeli Dostawcy jako ramki danych DataFrame
df_dostawcy = pd.read_sql_query("SELECT * FROM magazyny", conn)

# Zamknięcie połączenia z bazą danych
conn.close()

# Wyświetlenie pierwszych kilku wierszy ramki danych
print(df_dostawcy)

  id_magazyn                          nazwa_magazynu pojemnosc
0          1                          Magazyn główny    250000
1          2                     Magazyn kompletacji     25000
2          3           Magazyn krótkich dat ważności     10000
3          4                    Magazyn reklamacyjny       500
4          5                        Magazyn sortowni     10000
5          6            Magazyn towarów uszkodzonych       500
6          7  Magazyn weryfikacji I kontroli jakości      1500


# Wykaz tabel

In [3]:
import sqlite3

# Połączenie z bazą danych
conn = sqlite3.connect('db_zapasy.db')
cursor = conn.cursor()

# Zapytanie SQL pobierające nazwy wszystkich tabel w bazie danych
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")

# Pobranie wyników zapytania
tables = cursor.fetchall()

# Wyświetlenie nazw tabel
print("Dostępne tabele:")
for table in tables:
    print(table[0])

# Zamknięcie połączenia
conn.close()

Dostępne tabele:
Magazyny
StanyMagazynowe
Klienci
Zamowienia
ZamowieniaSzczegoly
Dostawy
DostawySzczegoly
Dostawcy
Produkty


# Czyszczenie tabeli

In [1]:
import sqlite3

def clear_dostawcy_table(conn):
    c = conn.cursor()
    c.execute("DELETE FROM Dostawcy")
    conn.commit()
    print("Tabela Dostawcy została wyczyszczona.")

# Połączenie z bazą danych SQLite
conn = sqlite3.connect('DB_ZAPASY.db')

# Wyczyść tabelę Dostawcy
clear_dostawcy_table(conn)

# Zamknięcie połączenia z bazą danych
conn.close()


Tabela Dostawcy została wyczyszczona.


In [4]:
# Wyciąganie danych

import pandas as pd
import sqlite3

# Połączenie z bazą danych SQLite
conn = sqlite3.connect('DB_ZAPASY.db')

# Pobranie danych z tabeli Dostawcy jako ramki danych DataFrame
df_dostawcy = pd.read_sql_query("SELECT * FROM magazyny", conn)

# Zamknięcie połączenia z bazą danych
conn.close()

# Wyświetlenie pierwszych kilku wierszy ramki danych
print(df_dostawcy.head())

   id_magazyn                 nazwa_magazynu  pojemnosc
0           1                 Magazyn główny     100000
1           2            Magazyn kompletacji       1000
2           3  Magazyn krótkich dat ważności       1000
3           4           Magazyn reklamacyjny        500
4           5               Magazyn sortowni       1000


In [43]:
import sqlite3
import pandas as pd

# Połączenie z bazą danych SQLite3
conn = sqlite3.connect('db_zapasy.db')

# Zapytanie SQL
query = """
SELECT 
    z.data_zamowienia,
    p.kod_produktu,
    p.nazwa_produktu,
    SUM(zs.ilosc) AS suma_sprzedazy
FROM 
    ZamowieniaSzczegoly zs
JOIN 
    Zamowienia z ON zs.id_zamowienia = z.id_zamowienia
JOIN 
    Produkty p ON zs.kod_produktu = p.kod_produktu
GROUP BY 
    z.data_zamowienia, p.kod_produktu, p.nazwa_produktu
ORDER BY 
    z.data_zamowienia ASC, suma_sprzedazy DESC;
"""

# Wykonanie zapytania i wczytanie wyników do ramki danych
df = pd.read_sql_query(query, conn)

# Wyświetlenie ramki danych
df.head()

# Zamknięcie połączenia z bazą danych
conn.close()


In [44]:
df

,data_zamowienia,kod_produktu,nazwa_produktu,suma_sprzedazy
0,2023-03-01 00:00:00,33324,"Femflora, kaps., 60 szt",90000
1,2023-03-01 00:00:00,32122,"Canpol Lovi,smocz,uspok,Botanic,girl,0-3,22/869x2",49250
2,2023-03-01 00:00:00,32357,"Omegamed Baby+D 6m+, kaps.twist-off, 30 szt",36864
3,2023-03-01 00:00:00,13093,"Famogast, 40 mg, tabl.powl., 60 szt",34992
4,2023-03-01 00:00:00,43210,"Reishi, kaps.,(Yango), 90 szt",34560
...,...,...,...,...
75372,2023-04-25 00:00:00,46154,Kambukka kubek termiczny etna cantaloupe 300ml,1
75373,2023-04-25 00:00:00,47929,Pozytywne konopie olejek konopny cbd 5%,1
75374,2023-04-25 00:00:00,48488,"Beauty Formulas,krem,nawilż.,z wit.C,rozjaśn.1...",1
75375,2023-04-25 00:00:00,53046,Carex original mydlo zapas 1000ml,1


In [28]:
import sqlite3
import pandas as pd

# Połączenie z bazą danych SQLite3
conn = sqlite3.connect('db_zapasy.db')

# Zapytanie SQL
query = """
WITH SumaSprzedazy AS (
    SELECT 
        strftime('%Y', z.data_zamowienia) AS rok,
        strftime('%m', z.data_zamowienia) AS miesiac,
        p.kod_produktu,
        p.nazwa_produktu,
        SUM(zs.ilosc) AS suma_sprzedazy
    FROM 
        ZamowieniaSzczegoly zs
    JOIN 
        Zamowienia z ON zs.id_zamowienia = z.id_zamowienia
    JOIN 
        Produkty p ON zs.kod_produktu = p.kod_produktu
    GROUP BY 
        rok, miesiac, p.kod_produktu, p.nazwa_produktu
),
ProcentSprzedazy AS (
    SELECT 
        rok,
        miesiac,
        kod_produktu,
        nazwa_produktu,
        suma_sprzedazy,
        SUM(suma_sprzedazy) OVER (PARTITION BY rok, miesiac) AS suma_sprzedazy_miesiaca,
        (suma_sprzedazy * 100.0 / SUM(suma_sprzedazy) OVER (PARTITION BY rok, miesiac)) AS procent_sprzedazy
    FROM 
        SumaSprzedazy
),
ABC AS (
    SELECT 
        rok,
        miesiac,
        kod_produktu,
        nazwa_produktu,
        suma_sprzedazy,
        procent_sprzedazy,
        NTILE(3) OVER (PARTITION BY rok, miesiac ORDER BY procent_sprzedazy DESC) AS abc_group
    FROM 
        ProcentSprzedazy
)
SELECT 
    rok,
    miesiac,
    kod_produktu,
    nazwa_produktu,
    suma_sprzedazy,
    procent_sprzedazy,
    CASE
        WHEN abc_group = 1 THEN 'A'
        WHEN abc_group = 2 THEN 'B'
        ELSE 'C'
    END AS analiza_abc
FROM 
    ABC
ORDER BY 
    rok ASC, miesiac ASC, procent_sprzedazy DESC;

"""

# Wykonanie zapytania i wczytanie wyników do ramki danych
df1 = pd.read_sql_query(query, conn)

# Wyświetlenie ramki danych
df1.head()

# Zamknięcie połączenia z bazą danych
conn.close()


In [49]:
import sqlite3
import pandas as pd

# Połączenie z bazą danych SQLite3
conn = sqlite3.connect('db_zapasy.db')

# Zapytanie SQL
query = """
SELECT 
    s.data_stanu AS Data_Stanu,
    p.nazwa_produktu AS Nazwa_Produktu,
    m.nazwa_magazynu AS Nazwa_Magazynu,
    s.ilosc_na_stanie AS Ilosc_Na_Stanie,
    s.ilosc_dostepna AS Ilosc_Dostepna,
    s.ilosc_zarezerwowana AS Ilosc_Zarezerwowana
FROM 
    StanyMagazynowe s
JOIN 
    Produkty p ON s.kod_produktu = p.kod_produktu
JOIN 
    Magazyny m ON s.id_magazyn = m.id_magazyn;
"""

# Wykonanie zapytania i wczytanie wyników do ramki danych
df1 = pd.read_sql_query(query, conn)

# Wyświetlenie ramki danych
df1.head()

# Zamknięcie połączenia z bazą danych
conn.close()


In [50]:
df1

,Data_Stanu,Nazwa_Produktu,Nazwa_Magazynu,Ilosc_Na_Stanie,Ilosc_Dostepna,Ilosc_Zarezerwowana
0,2023-05-12 00:00:00,"Zyrtec, 10 mg/ml, krople doustne, 20 ml",Magazyn główny,22,22,0
1,2023-05-12 00:00:00,"Linomag, 200 mg/g, krem, 30 g,tuba",Magazyn weryfikacji I kontroli jakości,7,7,0
2,2023-05-12 00:00:00,"Traumon, 100 mg/g, żel, 50 g",Magazyn weryfikacji I kontroli jakości,27,27,0
3,2023-05-12 00:00:00,"Olej rycynowy, płyn doustny, (Microfarm),100 g",Magazyn główny,65,65,0
4,2023-05-12 00:00:00,"Lewitan MOL, tabl., melisa, 100 g",Magazyn główny,3,3,0
...,...,...,...,...,...,...
234226,2023-05-18 00:00:00,"Bedicort G, (0,5 mg+1 mg)/g, maść, 30 g",Magazyn główny,26,26,0
234227,2023-05-18 00:00:00,"Dermitopic, 0,1%, maść, 10 g, tuba",Magazyn kompletacji,10,10,0
234228,2023-05-18 00:00:00,"Dermitopic, 0,1%, maść, 30 g, tuba",Magazyn główny,70,70,0
234229,2023-05-18 00:00:00,"Beta Karoten, 10000 IU, kaps., 100 szt",Magazyn główny,233,231,2


In [ ]:
import sqlite3
import pandas as pd
import streamlit as st
import io
import plotly.express as px

# Funkcja do pobierania danych z bazy danych na podstawie wybranej daty i magazynów
def get_data(selected_dates, selected_magazines):
    conn = sqlite3.connect('db_zapasy.db')
    dates_filter = "'" + "', '".join(selected_dates) + "'"
    magazines_filter = "'" + "', '".join(selected_magazines) + "'" if selected_magazines else "ALL"
    where_clause = f"DATE(s.data_stanu) IN ({dates_filter})"
    if selected_magazines:
        where_clause += f" AND m.nazwa_magazynu IN ({magazines_filter})"
    query = f"""
    SELECT 
        DATE(s.data_stanu) AS Data_Stanu,
        p.nazwa_produktu AS Nazwa_Produktu,
        m.nazwa_magazynu AS Nazwa_Magazynu,
        s.ilosc_dostepna AS Ilosc_Dostepna
    FROM 
        StanyMagazynowe s
    JOIN 
        Produkty p ON s.kod_produktu = p.kod_produktu
    JOIN 
        Magazyny m ON s.id_magazyn = m.id_magazyn
    WHERE 
        {where_clause};
    """
    df = pd.read_sql_query(query, conn)
    conn.close()
    return df

# Funkcja do pobierania danych tylko do wizualizacji wykresu
def get_chart_data(selected_dates, selected_magazines):
    conn = sqlite3.connect('db_zapasy.db')
    dates_filter = "'" + "', '".join(selected_dates) + "'"
    magazines_filter = "'" + "', '".join(selected_magazines) + "'" if selected_magazines else "ALL"
    where_clause = f"DATE(s.data_stanu) IN ({dates_filter})"
    if selected_magazines:
        where_clause += f" AND m.nazwa_magazynu IN ({magazines_filter})"
    query = f"""
    SELECT 
        DATE(s.data_stanu) AS Data_Stanu,
        m.nazwa_magazynu AS Nazwa_Magazynu,
        SUM(s.ilosc_dostepna) AS Ilosc_Dostepna
    FROM 
        StanyMagazynowe s
    JOIN 
        Magazyny m ON s.id_magazyn = m.id_magazyn
    WHERE 
        {where_clause}
    GROUP BY 
        DATE(s.data_stanu), m.nazwa_magazynu;
    """
    df = pd.read_sql_query(query, conn)
    conn.close()
    return df

# Pobranie unikalnych dat ze zbioru danych
conn = sqlite3.connect('db_zapasy.db')
dates_query = "SELECT DISTINCT DATE(data_stanu) AS data_stanu FROM StanyMagazynowe;"
dates = pd.read_sql_query(dates_query, conn)['data_stanu'].tolist()

# Pobranie unikalnych nazw magazynów
magazines_query = "SELECT DISTINCT nazwa_magazynu FROM Magazyny;"
magazines = pd.read_sql_query(magazines_query, conn)['nazwa_magazynu'].tolist()
conn.close()

# Pobranie najpóźniejszej dostępnej daty
latest_dates = [max(dates)]

# Dodanie tytułu "Aktualny stan zapasów" nad wyświetlanymi danymi
st.write("# Aktualny stan zapasów")

# Wybór daty za pomocą Multiselect Boxa, domyślnie ustawiony na najpóźniejszą dostępną datę
selected_dates = st.multiselect("Wybierz daty:", dates, default=latest_dates)

# Wybór magazynów za pomocą Multiselect Boxa
selected_magazines = st.multiselect("Wybierz magazyny:", magazines)

# Pobranie danych na podstawie wybranej daty i magazynów
df = get_data(selected_dates, selected_magazines)

# Wyświetlenie danych
st.write("Aktualny stan zapasów na wybrane daty i magazyny:")
st.write(df)

# Interaktywny wykres zmiany ilości dostępnych produktów w czasie
if not df.empty:
    chart_df = get_chart_data(selected_dates, selected_magazines)
    fig = px.line(chart_df, x='Data_Stanu', y='Ilosc_Dostepna', color='Nazwa_Magazynu', title='Zmiana ilości dostępnych produktów w czasie')
    fig.update_layout(xaxis_title='Data', yaxis_title='Dostępna ilość')
    fig.update_layout(xaxis=dict(tickmode='linear', dtick='D1'))
    st.plotly_chart(fig)

# Informacje o ilości produktów na magazynie
if selected_magazines:
    total_products = df.groupby('Nazwa_Magazynu')['Ilosc_Dostepna'].sum().to_dict()
    st.write("Łączna ilość produktów w zależności od wybranego magazynu:")
    for magazyn in selected_magazines:
        if magazyn in total_products:
            st.write(f"{magazyn}: {total_products[magazyn]} produktów")
else:
    st.write(f"Łączna ilość produktów: {df['Ilosc_Dostepna'].sum()}")

# Dodanie przycisku do pobierania danych jako pliku Excel
output = io.BytesIO()
with pd.ExcelWriter(output, engine='xlsxwriter') as writer:
     df.to_excel(writer, index=False, sheet_name='Sheet1')
output.seek(0)
st.download_button(
      label="Pobierz jako plik Excel",
      data=output.getvalue(),
      file_name="analiza_abc.xlsx",
     mime="application/vnd.openxmlformats-officedocument.spreadsheetml.sheet"
  )




In [29]:

# Połączenie z bazą danych SQLite3
conn = sqlite3.connect('db_zapasy.db')

# Zapytanie SQL
query = """
SELECT
    z.data_zamowienia AS 'Data zamówienia',
    p.kod_produktu,
    p.nazwa_produktu,
    SUM(zs.ilosc) AS suma_sprzedazy
FROM 
    ZamowieniaSzczegoly zs
JOIN 
    Zamowienia z ON zs.id_zamowienia = z.id_zamowienia
JOIN 
    Produkty p ON zs.kod_produktu = p.kod_produktu
GROUP BY 
    z.data_zamowienia, p.kod_produktu, p.nazwa_produktu
ORDER BY 
    suma_sprzedazy DESC;
"""

# Wykonanie zapytania i wczytanie wyników do ramki danych
df = pd.read_sql_query(query, conn)

# Zamknięcie połączenia z bazą danych
conn.close()



In [8]:
conn = sqlite3.connect('db_zapasy.db')

# Zapytanie SQL
query = """
SELECT p.nazwa_produktu AS NazwaProduktu, SUM(zs.ilosc) AS SumaSprzedazy
FROM Produkty p
JOIN ZamowieniaSzczegoly zs ON p.kod_produktu = zs.kod_produktu
GROUP BY p.nazwa_produktu
ORDER BY SumaSprzedazy DESC;
"""

# Wykonanie zapytania i wczytanie wyników do ramki danych
df = pd.read_sql_query(query, conn)

# Zamknięcie połączenia z bazą danych
conn.close()



In [9]:
df

,NazwaProduktu,SumaSprzedazy
0,"Dermedic Normacne Preventi,żel,antybakt,d/myci...",198996
1,"Pirolam Intima Vag, 500 mg, tabl.dopochw., 1 szt",198723
2,"Torba papierowa DOZ, 1 szt",198557
3,"YoungeVity Elixir, kaps., 30 szt",198080
4,"PregnaPLUS, kaps., 30 szt",198004
...,...,...
34077,Aa intymna natural 95% żel nawil/hig/intym.300ml,1
34078,Aa help płyn do kąp/żel pod prysz.2w1s.o.s.400ml,1
34079,Aa aloes100%sorbet dzien/noc48h nawilż/koj.50 ml,1
34080,"AROMAhelp,plast.,z olejk.eteryczn,pow.2 lat,5szt",1


In [28]:
df1

,rok,miesiac,kod_produktu,nazwa_produktu,suma_sprzedazy
0,2023,03,22215,"Dermedic Normacne Preventi,żel,antybakt,d/myci...",198960
1,2023,03,32336,"Torba papierowa DOZ, 1 szt",198053
2,2023,03,37139,NEONAIL LAMPA LED 18W/36,133416
3,2023,03,19773,"Fiorda, pastyl,d/ssania,o sm.czarn.porzeczki,3...",133080
4,2023,03,33324,"Femflora, kaps., 60 szt",93270
...,...,...,...,...,...
46341,2023,04,53046,Carex original mydlo zapas 1000ml,1
46342,2023,04,53122,"Jordan Change, końc.do szczot.,wymienne,soft,2szt",1
46343,2023,04,53291,"Arganicare Natur.Hairc,odżyw,naw,argan,shea,400ml",1
46344,2023,04,53483,"Weleda Baby,krem,d/tw.,z białą malwą,d/dzieci,...",1


In [3]:
df

,Data_Stanu,Nazwa_Produktu,Nazwa_Magazynu,Ilosc_Dostepna
0,2023-05-12,"Zyrtec, 10 mg/ml, krople doustne, 20 ml",Magazyn główny,22
1,2023-05-12,"Linomag, 200 mg/g, krem, 30 g,tuba",Magazyn weryfikacji I kontroli jakości,7
2,2023-05-12,"Traumon, 100 mg/g, żel, 50 g",Magazyn weryfikacji I kontroli jakości,27
3,2023-05-12,"Olej rycynowy, płyn doustny, (Microfarm),100 g",Magazyn główny,65
4,2023-05-12,"Lewitan MOL, tabl., melisa, 100 g",Magazyn główny,3
...,...,...,...,...
234226,2023-05-18,"Bedicort G, (0,5 mg+1 mg)/g, maść, 30 g",Magazyn główny,26
234227,2023-05-18,"Dermitopic, 0,1%, maść, 10 g, tuba",Magazyn kompletacji,10
234228,2023-05-18,"Dermitopic, 0,1%, maść, 30 g, tuba",Magazyn główny,70
234229,2023-05-18,"Beta Karoten, 10000 IU, kaps., 100 szt",Magazyn główny,231


In [4]:
conn = sqlite3.connect('db_zapasy.db')

# Zapytanie SQL
query = """
SELECT 
    m.nazwa_magazynu,
    (SUM(p.objetosc_sztuki * sm.ilosc_na_stanie) / m.pojemnosc) * 100 AS procent_wypelnienia
FROM 
    Magazyny m
INNER JOIN 
    StanyMagazynowe sm ON m.id_magazyn = sm.id_magazyn
INNER JOIN 
    Produkty p ON sm.kod_produktu = p.kod_produktu
GROUP BY 
    m.nazwa_magazynu, m.pojemnosc;

"""

# Wykonanie zapytania i wczytanie wyników do ramki danych
df = pd.read_sql_query(query, conn)

# Zamknięcie połączenia z bazą danych
conn.close()

In [5]:
df

,nazwa_magazynu,procent_wypelnienia
0,Magazyn główny,8023.729050
1,Magazyn kompletacji,712.416483
2,Magazyn krótkich dat ważności,418.829038
3,Magazyn reklamacyjny,14.363252
4,Magazyn sortowni,1663.480408
5,Magazyn towarów uszkodzonych,11.479479
6,Magazyn weryfikacji I kontroli jakości,6897.299902


In [8]:
conn = sqlite3.connect('db_zapasy.db')

# Zapytanie SQL
query = """
SELECT 
        m.nazwa_magazynu,
        CAST(SUM(p.objetosc_sztuki * sm.ilosc_na_stanie) AS INTEGER)
    FROM 
        Magazyny m
    INNER JOIN 
        StanyMagazynowe sm ON m.id_magazyn = sm.id_magazyn
    INNER JOIN 
        Produkty p ON sm.kod_produktu = p.kod_produktu
    WHERE
        DATE(sm.data_stanu) = '2023-05-18'
    GROUP BY 
        m.nazwa_magazynu, m.pojemnosc;
    """

# Wykonanie zapytania i wczytanie wyników do ramki danych
df = pd.read_sql_query(query, conn)

# Zamknięcie połączenia z bazą danych
conn.close()


In [9]:
df

,nazwa_magazynu,CAST(SUM(p.objetosc_sztuki * sm.ilosc_na_stanie) AS INTEGER)
0,Magazyn główny,2853914
1,Magazyn kompletacji,129869
2,Magazyn krótkich dat ważności,4687
3,Magazyn reklamacyjny,19
4,Magazyn sortowni,54755
5,Magazyn towarów uszkodzonych,37
6,Magazyn weryfikacji I kontroli jakości,3451


In [41]:
df

,nazwa_magazynu,procent_wypelnienia
0,Magazyn główny,20059.322626
1,Magazyn kompletacji,17810.412076
2,Magazyn krótkich dat ważności,4188.290377
3,Magazyn reklamacyjny,14.363252
4,Magazyn sortowni,16634.804079
5,Magazyn towarów uszkodzonych,11.479479
6,Magazyn weryfikacji I kontroli jakości,20691.899707


In [30]:
import sqlite3
import pandas as pd

conn = sqlite3.connect('db_zapasy.db')

# Zapytanie SQL
query = """
SELECT *
FROM Dostawcy
"""

# Wykonanie zapytania i wczytanie wyników do ramki danych
df = pd.read_sql_query(query, conn)

# Zamknięcie połączenia z bazą danych
conn.close()

In [31]:
df

,id_dostawca,nazwa_dostawcy
0,1,NatureBlend Pharmaceuticals
1,2,RenewSkin Laboratories
2,3,BellaVista
3,4,BeautyBoost
4,5,WellnessWise
...,...,...
358,359,DermAlive
359,360,SkinSavvy
360,361,Zenith Beauty
361,362,Pure Essence
